In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz
!tar -xvf spark-3.2.0-bin-hadoop2.7.tgz

!pip install -q findspark

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

from google.colab import files

spark-3.2.0-bin-hadoop2.7/
spark-3.2.0-bin-hadoop2.7/NOTICE
spark-3.2.0-bin-hadoop2.7/kubernetes/
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/python_executable_check.py
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/autoscale.py
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/worker_memory_check.py
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/py_container_checks.py
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/decommissioning.py
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/pyfiles.py
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/decommissioning_cleanup.py
spark-3.2.0-bin-hadoop2.7/kubernetes/dockerfiles/
spark-3.2.0-bin-hadoop2.7/kubernetes/dockerfiles/spark/
spark-3.2.0-bin-hadoop2.7/kubernetes/dockerfiles/spark/decom.sh
spark-3.2.0-bin-hadoop2.7/kubernetes/dockerfiles/spark/entrypoint.sh
spark-3.2.0-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/
spark-3.2.0-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/R/
spark-3.2.0-bin-hadoop2.7/kubernetes/docker

In [2]:
import pandas as pd

test_gr_truth = pd.read_csv('test2_new.txt', sep = '|', header=None)
test_gr_truth.columns = ['UserID', 'ItemID', 'Predictor']
test_gr_truth.head()

,UserID,ItemID,Predictor
0,200031,30877,1
1,200031,8244,1
2,200031,130183,0
3,200031,198762,0
4,200031,34503,1


In [3]:
three_ratings = pd.read_csv('output1.txt', sep='|', header=None)
three_ratings.columns = ['UserID', 'ItemID', 'Rating1', 'Rating2']
three_ratings.head()

,UserID,ItemID,Rating1,Rating2
0,199810,208019,0.0,0.0
1,199810,74139,0.0,0.0
2,199810,9903,0.0,0.0
3,199810,242681,0.0,0.0
4,199810,18515,0.0,70.0


In [4]:
test_gr_truth['Rating1'] = 0.0
test_gr_truth['Rating2'] = 0.0
len(three_ratings)
test_gr_truth = test_gr_truth[['UserID', 'ItemID', 'Rating1', 'Rating2','Predictor']]
test_gr_truth.head()

,UserID,ItemID,Rating1,Rating2,Predictor
0,200031,30877,0.0,0.0,1
1,200031,8244,0.0,0.0,1
2,200031,130183,0.0,0.0,0
3,200031,198762,0.0,0.0,0
4,200031,34503,0.0,0.0,1


In [5]:
for i in range(len(test_gr_truth)):
    user = test_gr_truth['UserID'][i]
    item = test_gr_truth['ItemID'][i]
    test_gr_truth['Rating1'][i] = three_ratings[three_ratings['UserID']==user][three_ratings['ItemID']==item]['Rating1']
    test_gr_truth['Rating2'][i] = three_ratings[three_ratings['UserID']==user][three_ratings['ItemID']==item]['Rating2']
    

流式输出内容被截断，只能显示最后 5000 行内容。
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

In [6]:
test_gr_truth.head(6)

,UserID,ItemID,Rating1,Rating2,Predictor
0,200031,30877,90.0,50.0,1
1,200031,8244,90.0,0.0,1
2,200031,130183,0.0,0.0,0
3,200031,198762,0.0,0.0,0
4,200031,34503,90.0,50.0,1
5,200031,227283,0.0,90.0,0


In [7]:
test_gr_truth.to_csv('ground_truth.csv', index=False)
three_ratings.to_csv('three_ratings.csv', index=False)

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('recommender').getOrCreate()
df1 = spark.read.csv('ground_truth.csv', header=True, inferSchema=True)
df1.printSchema()

root
 |-- UserID: integer (nullable = true)
 |-- ItemID: integer (nullable = true)
 |-- Rating1: double (nullable = true)
 |-- Rating2: double (nullable = true)
 |-- Predictor: integer (nullable = true)



In [9]:
df1.show(10)

+------+------+-------+-------+---------+
|UserID|ItemID|Rating1|Rating2|Predictor|
+------+------+-------+-------+---------+
|200031| 30877|   90.0|   50.0|        1|
|200031|  8244|   90.0|    0.0|        1|
|200031|130183|    0.0|    0.0|        0|
|200031|198762|    0.0|    0.0|        0|
|200031| 34503|   90.0|   50.0|        1|
|200031|227283|    0.0|   90.0|        0|
|200032|218377|    0.0|    0.0|        0|
|200032|110262|    0.0|    0.0|        0|
|200032| 18681|   90.0|   90.0|        1|
|200032|138493|   90.0|   90.0|        1|
+------+------+-------+-------+---------+
only showing top 10 rows



In [10]:
cols = df1.columns
from pyspark.ml.feature import VectorAssembler

assemblerInput = ['Rating1', 'Rating2']
assembler = VectorAssembler(inputCols = assemblerInput, outputCol='features')
stages=[]

stages += [assembler]
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(df1)
df1 = pipelineModel.transform(df1)
selectedCols = ['features'] + cols
df1 = df1.select(selectedCols)
df1.printSchema()
train, test = df1.randomSplit([0.7, 0.3], seed=2018)
print('Training Dataset Count: '+str(train.count()))
print('Test Dataset Count: '+str(test.count()))

root
 |-- features: vector (nullable = true)
 |-- UserID: integer (nullable = true)
 |-- ItemID: integer (nullable = true)
 |-- Rating1: double (nullable = true)
 |-- Rating2: double (nullable = true)
 |-- Predictor: integer (nullable = true)

Training Dataset Count: 4260
Test Dataset Count: 1740


In [11]:
type(df1)

pyspark.sql.dataframe.DataFrame

In [12]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'features', labelCol = 'Predictor', maxIter=10)
lrModel = lr.fit(df1)
trainingSummary = lrModel.summary
print("Training Set Area Under ROC: "+str(trainingSummary.areaUnderROC))
main_df = spark.read.csv('three_ratings.csv', header=True, inferSchema=True)
main_df.printSchema()
main_cols = main_df.columns
main_df = pipelineModel.transform(main_df)
selCols = ['features'] + main_cols
main_df = main_df.select(selCols)
main_df.printSchema()
predictions_120 = lrModel.transform(main_df)
predictions_120.show(10)

Training Set Area Under ROC: 0.8749316111111111
root
 |-- UserID: integer (nullable = true)
 |-- ItemID: integer (nullable = true)
 |-- Rating1: double (nullable = true)
 |-- Rating2: double (nullable = true)

root
 |-- features: vector (nullable = true)
 |-- UserID: integer (nullable = true)
 |-- ItemID: integer (nullable = true)
 |-- Rating1: double (nullable = true)
 |-- Rating2: double (nullable = true)

+-------------+------+------+-------+-------+--------------------+--------------------+----------+
|     features|UserID|ItemID|Rating1|Rating2|       rawPrediction|         probability|prediction|
+-------------+------+------+-------+-------+--------------------+--------------------+----------+
|    (2,[],[])|199810|208019|    0.0|    0.0|[1.42417015136285...|[0.80599133227115...|       0.0|
|    (2,[],[])|199810| 74139|    0.0|    0.0|[1.42417015136285...|[0.80599133227115...|       0.0|
|    (2,[],[])|199810|  9903|    0.0|    0.0|[1.42417015136285...|[0.80599133227115...|      

In [17]:
lr_answer = predictions_120.select('UserID','ItemID','prediction')
lr_answer = lr_answer.toPandas()
lr_answer['TrackID'] = ''
lr_answer.head()

,UserID,ItemID,prediction,TrackID
0,199810,208019,0.0,
1,199810,74139,0.0,
2,199810,9903,0.0,
3,199810,242681,0.0,
4,199810,18515,1.0,


In [18]:
for i in range(len(lr_answer)):
    lr_answer['TrackID'][i] = str(lr_answer['UserID'][i])+'_'+str(lr_answer['ItemID'][i])
lr_answer = lr_answer[['TrackID', 'UserID', 'ItemID', 'prediction']]
lr_answer = lr_answer.rename(columns={'prediction': 'Predictor'})
lr_answer.head()
lr_answer.drop(columns={'UserID', 'ItemID'}, inplace=True)
lr_answer.head()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,TrackID,Predictor
0,199810_208019,0.0
1,199810_74139,0.0
2,199810_9903,0.0
3,199810_242681,0.0
4,199810_18515,1.0


In [19]:
lr_answer.to_csv('lr_answer.csv', index=False)

In [20]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'Predictor', maxDepth=3)
dtModel = dt.fit(df1)
preds_120_dt = dtModel.transform(main_df)
preds_120_dt.columns

['features',
 'UserID',
 'ItemID',
 'Rating1',
 'Rating2',
 'rawPrediction',
 'probability',
 'prediction']

In [35]:
preds_120_dt.show(10)
dt_answer = preds_120_dt.select('UserID', 'ItemID', 'prediction')
dt_answer = dt_answer.toPandas()
dt_answer['TrackID'] = ''
dt_answer.head()

+-------------+------+------+-------+-------+--------------+--------------------+----------+
|     features|UserID|ItemID|Rating1|Rating2| rawPrediction|         probability|prediction|
+-------------+------+------+-------+-------+--------------+--------------------+----------+
|    (2,[],[])|199810|208019|    0.0|    0.0|[2775.0,649.0]|[0.81045560747663...|       0.0|
|    (2,[],[])|199810| 74139|    0.0|    0.0|[2775.0,649.0]|[0.81045560747663...|       0.0|
|    (2,[],[])|199810|  9903|    0.0|    0.0|[2775.0,649.0]|[0.81045560747663...|       0.0|
|    (2,[],[])|199810|242681|    0.0|    0.0|[2775.0,649.0]|[0.81045560747663...|       0.0|
|   [0.0,70.0]|199810| 18515|    0.0|   70.0|[224.0,2252.0]|[0.09046849757673...|       1.0|
|   [0.0,90.0]|199810|105760|    0.0|   90.0|[224.0,2252.0]|[0.09046849757673...|       1.0|
|    (2,[],[])|199812|276940|    0.0|    0.0|[2775.0,649.0]|[0.81045560747663...|       0.0|
|[100.0,100.0]|199812|142408|  100.0|  100.0|[224.0,2252.0]|[0.0904684

,UserID,ItemID,prediction,TrackID
0,199810,208019,0.0,
1,199810,74139,0.0,
2,199810,9903,0.0,
3,199810,242681,0.0,
4,199810,18515,1.0,


In [36]:
for i in range(len(dt_answer)):
    dt_answer['TrackID'][i] = str(dt_answer['UserID'][i])+'_'+str(dt_answer['ItemID'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
dt_answer = dt_answer[['TrackID', 'UserID', 'ItemID', 'prediction']]
dt_answer = dt_answer.rename(columns={'prediction': 'Predictor'})
dt_answer.head()


,TrackID,UserID,ItemID,Predictor
0,199810_208019,199810,208019,0.0
1,199810_74139,199810,74139,0.0
2,199810_9903,199810,9903,0.0
3,199810_242681,199810,242681,0.0
4,199810_18515,199810,18515,1.0


In [38]:
dt_answer.drop(columns={'UserID', 'ItemID'}, inplace=True)
dt_answer.head()

,TrackID,Predictor
0,199810_208019,0.0
1,199810_74139,0.0
2,199810_9903,0.0
3,199810_242681,0.0
4,199810_18515,1.0


In [39]:
dt_answer.to_csv('dt_answer.csv', index=False)

In [41]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol= 'features', labelCol='Predictor')
rfModel = rf.fit(df1)
preds_120_rf = rfModel.transform(main_df)
preds_120_rf.columns
preds_120_rf.show(10)

+-------------+------+------+-------+-------+--------------------+--------------------+----------+
|     features|UserID|ItemID|Rating1|Rating2|       rawPrediction|         probability|prediction|
+-------------+------+------+-------+-------+--------------------+--------------------+----------+
|    (2,[],[])|199810|208019|    0.0|    0.0|[16.2704262719815...|[0.81352131359907...|       0.0|
|    (2,[],[])|199810| 74139|    0.0|    0.0|[16.2704262719815...|[0.81352131359907...|       0.0|
|    (2,[],[])|199810|  9903|    0.0|    0.0|[16.2704262719815...|[0.81352131359907...|       0.0|
|    (2,[],[])|199810|242681|    0.0|    0.0|[16.2704262719815...|[0.81352131359907...|       0.0|
|   [0.0,70.0]|199810| 18515|    0.0|   70.0|[5.77903550272863...|[0.28895177513643...|       1.0|
|   [0.0,90.0]|199810|105760|    0.0|   90.0|[3.50734911722138...|[0.17536745586106...|       1.0|
|    (2,[],[])|199812|276940|    0.0|    0.0|[16.2704262719815...|[0.81352131359907...|       0.0|
|[100.0,10

In [42]:
rf_answer = preds_120_rf.select('UserID', 'ItemID', 'prediction')
rf_answer = rf_answer.toPandas()
rf_answer['TrackID'] = ''
rf_answer.head()

,UserID,ItemID,prediction,TrackID
0,199810,208019,0.0,
1,199810,74139,0.0,
2,199810,9903,0.0,
3,199810,242681,0.0,
4,199810,18515,1.0,


In [43]:
for i in range(len(rf_answer)):
    rf_answer['TrackID'][i] = str(rf_answer['UserID'][i])+'_'+str(rf_answer['ItemID'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
rf_answer = rf_answer[['TrackID', 'UserID', 'ItemID', 'prediction']]
rf_answer = rf_answer.rename(columns={'prediction': 'Predictor'})
rf_answer.head()

,TrackID,UserID,ItemID,Predictor
0,199810_208019,199810,208019,0.0
1,199810_74139,199810,74139,0.0
2,199810_9903,199810,9903,0.0
3,199810_242681,199810,242681,0.0
4,199810_18515,199810,18515,1.0


In [45]:
rf_answer.drop(columns={'UserID', 'ItemID'}, inplace=True)
rf_answer.head()

,TrackID,Predictor
0,199810_208019,0.0
1,199810_74139,0.0
2,199810_9903,0.0
3,199810_242681,0.0
4,199810_18515,1.0


In [46]:
rf_answer.to_csv('rf_answer.csv', index=False)

In [47]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(featuresCol = 'features', labelCol = 'Predictor', maxIter=10)
gbtModel = gbt.fit(df1)
preds_120_gbt = gbtModel.transform(main_df)
preds_120_gbt.columns

['features',
 'UserID',
 'ItemID',
 'Rating1',
 'Rating2',
 'rawPrediction',
 'probability',
 'prediction']

In [48]:
preds_120_gbt.show(10)

+-------------+------+------+-------+-------+--------------------+--------------------+----------+
|     features|UserID|ItemID|Rating1|Rating2|       rawPrediction|         probability|prediction|
+-------------+------+------+-------+-------+--------------------+--------------------+----------+
|    (2,[],[])|199810|208019|    0.0|    0.0|[0.69621496476798...|[0.80097988422856...|       0.0|
|    (2,[],[])|199810| 74139|    0.0|    0.0|[0.69621496476798...|[0.80097988422856...|       0.0|
|    (2,[],[])|199810|  9903|    0.0|    0.0|[0.69621496476798...|[0.80097988422856...|       0.0|
|    (2,[],[])|199810|242681|    0.0|    0.0|[0.69621496476798...|[0.80097988422856...|       0.0|
|   [0.0,70.0]|199810| 18515|    0.0|   70.0|[-0.3823633131076...|[0.31762094530726...|       1.0|
|   [0.0,90.0]|199810|105760|    0.0|   90.0|[-0.6812060990097...|[0.20384853659116...|       1.0|
|    (2,[],[])|199812|276940|    0.0|    0.0|[0.69621496476798...|[0.80097988422856...|       0.0|
|[100.0,10

In [49]:
gbt_answer = preds_120_gbt.select('UserID', 'ItemID', 'prediction')
gbt_answer = gbt_answer.toPandas()
gbt_answer['TrackID'] = ''
gbt_answer.head()

,UserID,ItemID,prediction,TrackID
0,199810,208019,0.0,
1,199810,74139,0.0,
2,199810,9903,0.0,
3,199810,242681,0.0,
4,199810,18515,1.0,


In [50]:
for i in range(len(gbt_answer)):
    gbt_answer['TrackID'][i] = str(gbt_answer['UserID'][i])+'_'+str(gbt_answer['ItemID'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [51]:
gbt_answer = gbt_answer[['TrackID', 'UserID', 'ItemID', 'prediction']]
gbt_answer = gbt_answer.rename(columns={'prediction': 'Predictor'})
gbt_answer.head()

,TrackID,UserID,ItemID,Predictor
0,199810_208019,199810,208019,0.0
1,199810_74139,199810,74139,0.0
2,199810_9903,199810,9903,0.0
3,199810_242681,199810,242681,0.0
4,199810_18515,199810,18515,1.0


In [52]:
gbt_answer.drop(columns={'UserID', 'ItemID'}, inplace=True)
gbt_answer.head()

,TrackID,Predictor
0,199810_208019,0.0
1,199810_74139,0.0
2,199810_9903,0.0
3,199810_242681,0.0
4,199810_18515,1.0


In [53]:
gbt_answer.to_csv('gbt_answer.csv', index=False)